# Recreate the important sections of Part 1 and Part 2 of the assignment:

In [130]:
import folium

In [131]:
import pandas as pd
import numpy as np
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from IPython.display import Image
from IPython.core.display import HTML
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [132]:
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

df = data[0]

In [133]:
#Clean the dataframe and drop the Boroughs that have a value equal to "Not assigned"

df.drop(df[df['Borough'] == 'Not assigned'].index, inplace = True) 

In [135]:
new_df = pd.merge(df, dfwcoord, on='Postal Code')
new_df.head(3)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636


# Part 3 

## Explore and cluster the neighborhoods in Toronto

In [136]:
toronto_data = new_df[new_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


## Use Latitude and Longitude for Regent Park, Harbourfront area of downtown Toronto

In [138]:
latitude = 43.654260
longitude = -79.360636

## Search for Italian food within a 800 meters

In [139]:
search_query = 'Italian'
radius = 800

In [147]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [149]:
results = requests.get(url).json()

In [151]:
# assign relevant part of JSON to venues
venues = results['response']['venues']
# tranform venues into a dataframe
dataframe = json_normalize(venues)

## Show the Italian restaurants in the area

In [153]:
# keep only columns that include venue name, and anything that is associated with location

filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row

dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term

dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Vicino Italian Kitchen,Italian Restaurant,146 Sumach St,CA,Toronto,Canada,St David St.,562,"[146 Sumach St (St David St.), Toronto ON M5A ...","[{'label': 'display', 'lat': 43.659291, 'lng':...",43.659291,-79.360015,M5A 0P7,ON,5d4f546e0e971c0007dd24ff
1,Il Ponte Cucina Italiana,Italian Restaurant,625 Queen St East,CA,Toronto,Canada,NaN,736,"[625 Queen St East, Toronto ON M4M 1G7, Canada]","[{'label': 'display', 'lat': 43.65785164410602...",43.657852,-79.352961,M4M 1G7,ON,579f0363498ed2eec32ab828


In [154]:
dataframe_filtered.name

0      Vicino Italian Kitchen
1    Il Ponte Cucina Italiana
Name: name, dtype: object

## Show the Italian restaurants on a map

In [155]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel


# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map